In [1]:
% run utils.ipynb

In [2]:
import json
import io

In [3]:
ety = pd.read_csv('../data/etymwn.tsv', sep='\t', header=None)
ety.columns = ['src', 'rel', 'to']

ety = ety.assign(
    src_lang=ety.src.apply(lambda x: x.split(':')[0].strip()),
    src_word=ety.src.apply(lambda x: x.split(':')[1].strip()),
    to_lang=ety.to.apply(lambda x: x.split(':')[0].strip()),
    to_word=ety.to.apply(lambda x: x.split(':')[1].strip()),
)

ety.head()

,src,rel,to,src_lang,src_word,to_lang,to_word
0,aaq: Pawanobskewi,rel:etymological_origin_of,eng: Penobscot,aaq,Pawanobskewi,eng,Penobscot
1,aaq: senabe,rel:etymological_origin_of,eng: sannup,aaq,senabe,eng,sannup
2,abe: waniigan,rel:etymological_origin_of,eng: wangan,abe,waniigan,eng,wangan
3,abe: waniigan,rel:etymological_origin_of,eng: wannigan,abe,waniigan,eng,wannigan
4,abs: beta,rel:etymological_origin_of,zsm: beta,abs,beta,zsm,beta


In [28]:
word_lang = {}

for i, row in tqdm(ety.iterrows()):
    w = row.src_word.lower()
    if w not in word_lang:
        word_lang[w] = []
    if row.src_lang not in word_lang[w]:
        word_lang[w].append(row.src_lang)

6031431it [09:05, 11062.54it/s]


In [45]:
with open('../data/word_lang.json', 'w') as f:
    json.dump(word_lang, f, ensure_ascii=False)

In [31]:
langs = ety.src_lang.unique()
langs.sort()
langs.shape

(397,)

In [73]:
network_to = {}

for l in tqdm(langs):
    cnt = ety[ety.src_lang == l].groupby('to_lang').to_lang.count()
    network_to[l] = list(zip(cnt.index, map(int, cnt.values)))

100%|██████████| 397/397 [03:12<00:00,  2.06it/s]


In [74]:
network_to

{'aaq': [('eng', 2)],
 'abe': [('eng', 2)],
 'abs': [('zsm', 1)],
 'adt': [('eng', 1)],
 'afr': [('afr', 658),
  ('ara', 1),
  ('deu', 3),
  ('eng', 44),
  ('fin', 1),
  ('fra', 2),
  ('ind', 1),
  ('ita', 1),
  ('msa', 3),
  ('nld', 236),
  ('por', 2),
  ('rus', 1),
  ('spa', 1)],
 'aii': [('aii', 2),
  ('ara', 2),
  ('eng', 3),
  ('fra', 1),
  ('grc', 10),
  ('heb', 9),
  ('lat', 1),
  ('syc', 2)],
 'ain': [('ain', 18), ('jpn', 5), ('por', 1)],
 'akk': [('akk', 40),
  ('ara', 1),
  ('eng', 8),
  ('fas', 1),
  ('grc', 2),
  ('heb', 1),
  ('por', 1),
  ('rus', 1),
  ('sux', 9),
  ('syc', 26),
  ('xcl', 2)],
 'akz': [('akz', 4), ('eng', 3), ('mod', 1), ('spa', 1)],
 'ale': [('eng', 3), ('rus', 2)],
 'alq': [('eng', 1), ('fra', 1)],
 'amh': [('amh', 12), ('eng', 6), ('fra', 2), ('pol', 1)],
 'amj': [('eng', 1)],
 'ang': [('ang', 5823),
  ('dan', 2),
  ('eng', 1050),
  ('enm', 1389),
  ('epo', 1),
  ('fao', 2),
  ('fra', 4),
  ('fro', 14),
  ('gla', 2),
  ('gle', 3),
  ('glv', 2),
  ('gml

In [75]:
with open('../data/network_to.json', 'w') as f:
    json.dump(network_to, f, ensure_ascii=False)

In [76]:
network_from = {}

for l in tqdm(langs):
    cnt = ety[ety.to_lang == l].groupby('src_lang').src_lang.count()
    network_from[l] = list(zip(cnt.index, map(int, cnt.values)))

100%|██████████| 397/397 [03:20<00:00,  1.98it/s]


In [77]:
with open('../data/network_from.json', 'w') as f:
    json.dump(network_from, f, ensure_ascii=False)